In [310]:
import pandas as pd
import re
from tqdm.notebook import tqdm
import numpy as np
import plotly.express as px

# Locais de Votação

In [169]:
df_local = pd.read_csv('boca_urna/local_votacao.csv',delimiter='\t')
df_geo_bairro = pd.read_csv('boca_urna/geoloc.csv')

In [209]:
df_zona_1 = df_local[df_local['Zona'] == 1]
df_zona_1 = df_zona_1[['Bairro','Local']]
df_zona_1['Local'] = df_zona_1['Local'].apply(lambda x: str(x))
dict_secoes_zona_1 = df_zona_1.set_index('Local').to_dict()['Bairro']

In [224]:
# - Dicionários para traduzir bairro por latitude e longitude
dic_bairro_longitude  = pd.Series(df_geo_bairro.Longitude.values,index=df_geo_bairro.Bairro).to_dict()
dic_bairro_latitude  = pd.Series(df_geo_bairro.Latitude.values,index=df_geo_bairro.Bairro).to_dict()

# Votos

In [333]:
import os
directory = 'boca_urna/zona1/'
dict_zona1 = {}
lst_df_prefeito = []
for filename in tqdm(os.listdir(directory)):
    f = open(directory+filename, "r",encoding='utf-8')
    section = f.readlines()
    
    # - Identifica frases delimitadoras do texto
    eleitores_prefeito = section.index("Eleitores\n")
    section[eleitores_prefeito] = ("EleitoresPrefeito\n")

    info_indetificacao = 0
    info_urna = section.index("Urna Eletrônica\n")
    info_prefeito = section.index("Prefeito\n")
    info_eleitores_prefeito = section.index("EleitoresPrefeito\n")
    info_vereador= section.index("Vereador\n")
    info_eleitores = section.index("Eleitores\n")

    # - Separa em listas de conteúdo
    list_identificacao  = section[info_indetificacao:info_urna]
    list_urna = section[info_urna:info_prefeito]
    list_prefeito = section[info_prefeito+1:info_eleitores_prefeito]
    list_eleitores_prefeito = section[info_eleitores_prefeito:info_vereador]
    list_vereador = section[info_vereador+1:info_eleitores]
    list_eleitores = section[info_eleitores:]

    # - Identifica local de votação
    local_index = list_identificacao.index("Local de votação\n") + 1
    local_id = list_identificacao[local_index].strip('\n')
    
    # - Remove simbolos desnecessários
    list_prefeito = [item.strip('\n') for item in list_prefeito]
    list_vereador = [item.strip('\n') for item in list_vereador]
    
    # - Separa conteúdo do resultado
    N=4
    list_prefeito = [list_prefeito[n:n+N] for n in range(0, len(list_prefeito), N)]
    list_vereador = [list_vereador[n:n+N] for n in range(0, len(list_vereador), N)]
    
    # - Cria dicionário da votação para prefeito
    dict_prefeito = {}
    for item in list_prefeito:
        dict_prefeito[(item[0],item[1])] = item[3]
    
    df_prefeito = pd.DataFrame(dict_prefeito, index=[0]).T
    df_prefeito['local_id'] = local_id
    lst_df_prefeito.append(df_prefeito)
    
    # - Cria dicionário da votação para vereador
    dict_vereador = {}
    for item in list_vereador:
        dict_vereador[(item[0],item[1])] = item[3]
    
    dict_zona1[local_id] = [dict_prefeito,dict_vereador]
    

# Resultado

In [374]:
# - Concatena resultados da zona (Só concatenar a lista de todas as zonas e voilá)
df_prefeito = pd.concat(lst_df_prefeito)

# - Organiza dataframe da zona
df_prefeito.reset_index(inplace=True)
df_prefeito.columns = ['candidato','partido','num_votos','local_id']
df_prefeito['num_votos'] = df_prefeito['num_votos'].apply(lambda x: int(x))

# - Agrupa resultados
df_resultado = df_prefeito.groupby(['candidato', 'local_id']).agg('num_votos').sum()
df_resultado = df_resultado.reset_index()

# - Adiciona latitude e longitude
df_resultado['local_id'] = df_resultado['local_id'].map(dict_secoes_zona_1)
df_resultado['latitude'] = df_resultado['local_id'].map(dic_bairro_latitude)
df_resultado['longitude'] = df_resultado['local_id'].map(dic_bairro_longitude)

In [385]:
df_resultado.head()

,candidato,local_id,num_votos,latitude,longitude
0,11 - CICERO LUCENA,CENTRO,104,-7.120323,-34.881876
1,11 - CICERO LUCENA,CENTRO,265,-7.120323,-34.881876
2,11 - CICERO LUCENA,CENTRO,66,-7.120323,-34.881876
3,11 - CICERO LUCENA,NaN,284,NaN,NaN
4,11 - CICERO LUCENA,CENTRO,241,-7.120323,-34.881876


# Plot

In [386]:
fig_map = px.scatter_mapbox(df_resultado, lat="latitude", lon="longitude", color="candidato", size="num_votos",text ="local_id",
                  color_continuous_scale=px.colors.cyclical.IceFire, size_max=35, zoom=10,
                  mapbox_style="carto-positron")

In [387]:
fig_map